In [28]:
%%bash

# Download Example Data --------------------------------------------------------

rm -rf ./sample_data/

mkdir -p data

rm -f ./data/*

curl https://raw.githubusercontent.com/rschwess/tutorial_dl_for_genomics/master/data/pwm_seq_200bp_test_set.txt >./data/pwm_seq_200bp_test_set.txt
curl https://raw.githubusercontent.com/rschwess/tutorial_dl_for_genomics/master/data/pwm_seq_200bp_valid_set.txt >./data/pwm_seq_200bp_valid_set.txt
curl https://raw.githubusercontent.com/rschwess/tutorial_dl_for_genomics/master/data/pwm_seq_200bp_train_set.txt >./data/pwm_seq_200bp_train_set.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  198k  100  198k    0     0   558k      0 --:--:-- --:--:-- --:--:--  560k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  198k  100  198k    0     0   544k      0 --:--:-- --:--:-- --:--:--  546k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7533k  100 7533k    0     0  14.3M      0 --:--:-- --:--:-- --:--:-- 14.3M


In [29]:
%%bash

# Create some directories and download helper scripts for later ----------------
mkdir -p helper visualize

curl https://raw.githubusercontent.com/rschwess/tutorial_dl_for_genomics/master/helper/functions_for_motif_plotting.R >./helper/functions_for_motif_plotting.R
curl https://raw.githubusercontent.com/rschwess/tutorial_dl_for_genomics/master/helper/plot_sequence_kernel_weights_per_dir.R >./helper/plot_sequence_kernel_weights_per_dir.R
curl https://raw.githubusercontent.com/rschwess/tutorial_dl_for_genomics/master/helper/plot_sequence_kernel_icms_per_dir.R >./helper/plot_sequence_kernel_icms_per_dir.R


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18139  100 18139    0     0  67770      0 --:--:-- --:--:-- --:--:-- 67936
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1512  100  1512    0     0   5306      0 --:--:-- --:--:-- --:--:--  5305
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1510  100  1510    0     0   5725      0 --:--:-- --:--:-- --:--:--  5741


In [30]:
%%bash

# Check Data Layout ###########################

# check lines per set
echo "Numbers:"
wc -l ./data/pwm*

# check data format
echo -ne "\nFormat\n"
head -n 3 ./data/pwm_seq_200bp_test_set.txt

# check class representation
echo -ne "\nClass Representations:\n"

echo -ne "\nTraining:\n"
cut -f 1 ./data/pwm_seq_200bp_train_set.txt | sort | uniq -c
echo -ne "\nTest:\n"
cut -f 1 ./data/pwm_seq_200bp_test_set.txt | sort | uniq -c
echo -ne "\nValidation:\n"
cut -f 1 ./data/pwm_seq_200bp_valid_set.txt | sort | uniq -c



Numbers:
   1000 ./data/pwm_seq_200bp_test_set.txt
  38000 ./data/pwm_seq_200bp_train_set.txt
   1000 ./data/pwm_seq_200bp_valid_set.txt
  40000 total

Format
3	ATGGCTGATAATGACGATTGTACAGATGGTGGATGAGATTGCCTCGTCCCGGCAGCATTACCCCCTGGTGGCAACGGCCACCAGGGGGCAATAAATCTGTGTCTTATCTCCGAGACCAAACAATTCCACAGCCTCTTATACAGCACCGAATGGACCGCCCCCTGGTGGCCAGGTATCGTCGAGGGCTCAATTAAACTCCT
1	GCAGGCATTATGAGGTAATAAACTCAGCGCGTGTTGAGATAAGATTCTAAGCGGCGCGCGCGCGCGACCGCGAGAAGTGGAGATTAAGCGCGCTAATGGTGTGTCCGATAGTCACGTGTCCGCGCGGCGCGCGCCATGTATGTTCTGTTCTGCGCGCCGCGCTTTGCGCGCGCGCTTGGTATATAAAGCTGGGTTTTAAT
1	GGCGCGCCTGGCATTTCTTAGAGAGGCGCGCAATACAACGAGAATCACCTAGAAGCCGTGTCTGTTGCTTATCACCGTTCGCCTAGGCCGCACGGGCACGTGGGTCTCCCGTTCCCTCAATCCTAACAGAAGCGCGCTAAGTCGTCGTTGGCTCTCTTACTAGCAGCGCGCCTGTACTAACCCGGCACTCGGCGGTGGGC

Class Representations:

Training:
   9489 0
   9513 1
   9508 2
   9490 3

Test:
    268 0
    237 1
    243 2
    252 3

Validation:
    243 0
    250 1
    249 2
    258 3


In [1]:
pip install hmmlearn tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from hmmlearn import hmm
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [5]:
def encode_sequence(seq):
    return [nucleotide_map[nuc] for nuc in seq if nuc in nucleotide_map]

def load_data(filepath):
    data = pd.read_csv(filepath, sep="\t", header=None, names=["label", "sequence"])
    data["encoded"] = data["sequence"].apply(encode_sequence)
    return data

def train_hmm_per_label(train_data, n_states=4):
    label_hmms = {}
    for label in sorted(train_data['label'].unique()):
        label_data = train_data[train_data['label'] == label]
        sequences = label_data['encoded'].tolist()
        lengths = [len(seq) for seq in sequences]
        X = np.concatenate([np.array(seq) for seq in sequences]).reshape(-1, 1)

        model = hmm.CategoricalHMM(n_components=n_states, n_iter=200, tol=1e-5, init_params='ste',algorithm='viterbi', verbose= True)
        model.fit(X, lengths)
        label_hmms[label] = model
    return label_hmms

def predict_labels(hmm_models, data):
    predictions = []
    for seq in data['encoded']:
        seq_array = np.array(seq).reshape(-1, 1)
        scores = {label: model.score(seq_array) for label, model in hmm_models.items()}
        pred_label = max(scores, key=scores.get)
        predictions.append(pred_label)
    return predictions

In [7]:
def evaluate(true_labels, pred_labels, set_name="Test"):
    print(f"\n--- {set_name} Set Evaluation ---")
    print("Accuracy:", accuracy_score(true_labels, pred_labels))
    print("Confusion Matrix:\n", confusion_matrix(true_labels, pred_labels))
    print("Classification Report:\n", classification_report(true_labels, pred_labels))


In [9]:
nucleotide_map = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

In [13]:
# Load data
train = load_data("data/pwm_seq_200bp_train_set.txt")
test = load_data("data/pwm_seq_200bp_test_set.txt")
valid = load_data("data/pwm_seq_200bp_valid_set.txt")


In [17]:
train['label']

Series([], Name: label, dtype: object)

In [37]:
# Train HMMs
hmm_models = train_hmm_per_label(train, n_states=4)

         1 -2951928.00514724             +nan
         2 -2465597.15258389 +486330.85256335
         3 -2446542.84863426  +19054.30394962
         4 -2435348.05847637  +11194.79015790
         5 -2428094.72701203   +7253.33146434
         6 -2423232.43042919   +4862.29658285
         7 -2419915.17516812   +3317.25526107
         8 -2417611.33159968   +2303.84356844
         9 -2415965.90507997   +1645.42651971
        10 -2414738.82370145   +1227.08137852
        11 -2413769.99731044    +968.82639101
        12 -2412954.90117247    +815.09613797
        13 -2412226.29619168    +728.60498079
        14 -2411540.92629555    +685.36989612
        15 -2410870.33011024    +670.59618532
        16 -2410194.75711935    +675.57299088
        17 -2409499.21493892    +695.54218044
        18 -2408770.85914697    +728.35579195
        19 -2407997.14424684    +773.71490012
        20 -2407164.31355333    +832.83069352
        21 -2406255.89588778    +908.41766554
        22 -2405250.89483015   +10

In [38]:
hmm_models[1].startprob_, hmm_models[1].emissionprob_ , hmm_models[1].transmat_

(array([0.0454999 , 0.01293222, 0.08944675, 0.85212114]),
 array([[4.66509942e-03, 3.61890704e-04, 9.94014974e-01, 9.58035988e-04],
        [9.17579708e-04, 9.92107117e-01, 2.64072790e-04, 6.71123085e-03],
        [4.29240019e-01, 6.28171011e-02, 3.97988217e-01, 1.09954664e-01],
        [2.38634415e-01, 2.61834421e-01, 2.19293125e-01, 2.80238038e-01]]),
 array([[1.96834780e-02, 9.65297707e-01, 9.09564559e-05, 1.49278582e-02],
        [5.80463526e-01, 2.05075199e-02, 5.92193833e-02, 3.39809570e-01],
        [4.37522558e-02, 1.47606064e-02, 4.27350191e-03, 9.37213636e-01],
        [3.61498226e-02, 1.90168739e-03, 1.24792641e-01, 8.37155849e-01]]))

In [39]:
hmm_models[0].startprob_, hmm_models[0].emissionprob_ , hmm_models[0].transmat_

(array([0.5514024 , 0.0722631 , 0.00548633, 0.37084817]),
 array([[2.63130246e-01, 2.43619816e-01, 2.28760405e-01, 2.64489533e-01],
        [4.94722907e-05, 3.75026723e-04, 9.99197915e-01, 3.77586376e-04],
        [8.04826239e-06, 9.99944567e-01, 1.47743063e-05, 3.26101748e-05],
        [2.63185099e-01, 2.24916745e-01, 2.53995145e-01, 2.57903011e-01]]),
 array([[6.08916001e-01, 7.06107551e-02, 1.56454256e-03, 3.18908702e-01],
        [1.53754737e-09, 3.45623858e-02, 9.56612724e-01, 8.82488894e-03],
        [3.12747256e-01, 6.53897904e-01, 3.32042218e-02, 1.50618832e-04],
        [6.01528217e-01, 1.08178787e-01, 6.29955250e-04, 2.89663041e-01]]))

In [40]:
hmm_models[2].startprob_, hmm_models[2].emissionprob_ , hmm_models[2].transmat_

(array([0.55490359, 0.01384896, 0.05536564, 0.37588181]),
 array([[1.54421375e-01, 3.47584536e-01, 3.69008599e-01, 1.28985490e-01],
        [3.39643825e-03, 1.72741562e-03, 4.30600638e-04, 9.94445545e-01],
        [9.94408871e-01, 1.30727691e-03, 9.80985714e-04, 3.30286626e-03],
        [3.60081391e-01, 1.41393399e-01, 9.78953473e-02, 4.00629862e-01]]),
 array([[5.70714481e-01, 2.67426116e-03, 3.24646414e-02, 3.94146617e-01],
        [2.15862829e-01, 2.19795464e-02, 5.89449307e-01, 1.72708318e-01],
        [5.46882055e-04, 9.48359069e-01, 2.39173191e-02, 2.71767295e-02],
        [7.87827751e-01, 1.06716186e-02, 3.91650303e-02, 1.62335600e-01]]))

In [41]:
hmm_models[3].startprob_, hmm_models[3].emissionprob_ , hmm_models[3].transmat_

(array([0.46464778, 0.23790438, 0.11815768, 0.17929016]),
 array([[0.27532449, 0.32138821, 0.21038071, 0.19290658],
        [0.36724511, 0.10288487, 0.27073018, 0.25913983],
        [0.11815867, 0.30213123, 0.08095468, 0.49875542],
        [0.12307696, 0.23320854, 0.45683493, 0.18687956]]),
 array([[3.88301563e-01, 3.41445148e-01, 2.70226369e-01, 2.69206240e-05],
        [2.36669978e-05, 5.95501482e-02, 3.51398050e-04, 9.40074787e-01],
        [9.81247562e-01, 2.35909302e-04, 1.85152858e-02, 1.24243612e-06],
        [7.83968203e-01, 1.96552750e-01, 7.94882797e-03, 1.15302189e-02]]))

In [42]:
# Predict
test_preds = predict_labels(hmm_models, test)
valid_preds = predict_labels(hmm_models, valid)

In [43]:
# Evaluate
evaluate(test['label'], test_preds, "Test")
evaluate(valid['label'], valid_preds, "Validation")


--- Test Set Evaluation ---
Accuracy: 0.906
Confusion Matrix:
 [[220  48   0   0]
 [ 20 211   0   6]
 [  0   0 237   6]
 [  0   8   6 238]]
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.82      0.87       268
           1       0.79      0.89      0.84       237
           2       0.98      0.98      0.98       243
           3       0.95      0.94      0.95       252

    accuracy                           0.91      1000
   macro avg       0.91      0.91      0.91      1000
weighted avg       0.91      0.91      0.91      1000


--- Validation Set Evaluation ---
Accuracy: 0.902
Confusion Matrix:
 [[195  48   0   0]
 [ 23 222   1   4]
 [  0   0 246   3]
 [  0  10   9 239]]
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.80      0.85       243
           1       0.79      0.89      0.84       250
           2       0.96      0.99      0.97       249
           3

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Load sequences and labels
def load_data(filepath):
    sequences, labels = [], []
    with open(filepath, 'r') as f:
        for line in f:
            label, seq = line.strip().split('\t')
            sequences.append(seq)
            labels.append(int(label))
    return sequences, labels

def one_hot_encode(seq, seq_len=200):
    mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    one_hot = np.zeros((seq_len, 4))
    for i, base in enumerate(seq):
        if base in mapping:
            one_hot[i, mapping[base]] = 1
    return one_hot

train_sequences, train_labels = load_data("/content/data/pwm_seq_200bp_train_set.txt")
test_sequences, test_labels = load_data("/content/data/pwm_seq_200bp_test_set.txt")

X_train = np.array([one_hot_encode(seq) for seq in train_sequences])
X_test = np.array([one_hot_encode(seq) for seq in test_sequences])
y_train = to_categorical(train_labels, num_classes=4)
y_test = to_categorical(test_labels, num_classes=4)

# CNN Model
model = Sequential([
    Conv1D(64, 10, activation='relu', input_shape=(200, 4)),
    MaxPooling1D(pool_size=3),
    Dropout(0.3),
    Conv1D(128, 10, activation='relu'),
    MaxPooling1D(pool_size=3),
    Dropout(0.3),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.1)

# Evaluate
preds = model.predict(X_test)
y_pred_cnn = np.argmax(preds, axis=1)
y_true_cnn = np.argmax(y_test, axis=1)
print("\nCNN Classification Report:\n")
print(classification_report(y_true_cnn, y_pred_cnn))

In [ ]:
#comparing model performance
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# HMM Accuracies (based on test and valid dataframes)
hmm_test_accuracy = accuracy_score(test['label'], test_preds)
hmm_valid_accuracy = accuracy_score(valid['label'], valid_preds)

# CNN Accuracies (from earlier code)
cnn_test_accuracy = accuracy_score(y_true_cnn, y_pred_cnn)

# Plot
labels = ['Test']
x = range(len(labels))
width = 0.35

plt.figure(figsize=(8, 4))
plt.bar([i - width/2 for i in x], [cnn_test_accuracy], width=width, label='CNN', color='green')
plt.bar([i + width/2 for i in x], [hmm_test_accuracy], width=width, label='HMM', color='red')

plt.xticks(x, labels)
plt.ylim(0, 1)
plt.ylabel('Accuracy')
plt.title('CNN vs HMM Accuracy: Test Set')
plt.legend()

# Annotate
for i, (c_acc, h_acc) in enumerate(zip([cnn_test_accuracy], [hmm_test_accuracy])):
    plt.text(i - width/2, c_acc + 0.02, f'{c_acc:.2f}', ha='center')
    plt.text(i + width/2, h_acc + 0.02, f'{h_acc:.2f}', ha='center')

plt.grid(axis='y')
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

# CNN Confusion Matrix
cm_cnn = confusion_matrix(y_true_cnn, y_pred_cnn)
cm_cnn_df = pd.DataFrame(cm_cnn, index=[f"True {i}" for i in range(4)],
                         columns=[f"Pred {i}" for i in range(4)])
print(" CNN Confusion Matrix:")
print(cm_cnn_df)

# HMM Confusion Matrix
cm_hmm = confusion_matrix(test['label'], test_preds)
cm_hmm_df = pd.DataFrame(cm_hmm, index=[f"True {i}" for i in range(4)],
                         columns=[f"Pred {i}" for i in range(4)])
print("\n HMM Confusion Matrix:")
print(cm_hmm_df)


In [ ]:
test_file = "/content/data/pwm_seq_200bp_test_set.txt"

In [ ]:
with open(test_file, "r") as f:
    seqs = []
    labels = []
    for line in f:
        line = line.rstrip().split("\t")
        seqs.append(line[1])
        labels.append(line[0])

# Select a single test sequence and its true label
single_seq = seqs[10]
single_label = labels[10]

print("🧬 Sequence:\n", single_seq)

# One-hot encode or integer-encode the sequence
encoded_seq = np.array(encode_sequence(single_seq)).reshape(-1, 1)

# ------------------------------------------------------------------
# Evaluate log-likelihood on each class-specific HMM model
log_likelihoods = {}
for cls, model in hmm_models.items():
    try:
        log_prob = model.score(encoded_seq)
    except:
        log_prob = -np.inf  # In case model can't process this sequence
    log_likelihoods[cls] = log_prob

# ------------------------------------------------------------------
# Display log-likelihoods
print("\n🔍 Class Prediction Log-Likelihoods:")
for cls in sorted(log_likelihoods.keys()):
    print(f"\tClass {cls} = {log_likelihoods[cls]:.2f}")

# Predict the class with highest likelihood
predicted_class = max(log_likelihoods, key=log_likelihoods.get)
print(f"\n✅ Predicted Class: {predicted_class}")
print(f"🎯 True Class: {single_label}")
